<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [ ]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [514]:
!mkdir '/content/waipio2022'
!mkdir '/content/waipio2022/Images'
!mkdir '/content/waipio2022/Labels'

In [515]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/naip/waipio2022.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/naip/waipio2022.zip
   creating: /content/CCAP/content/waipio2022/Images/
  inflating: /content/CCAP/content/waipio2022/Images/1_1.tif  
  inflating: /content/CCAP/content/waipio2022/Images/1_513.tif  
  inflating: /content/CCAP/content/waipio2022/Images/1_797.tif  
  inflating: /content/CCAP/content/waipio2022/Images/226_1.tif  
  inflating: /content/CCAP/content/waipio2022/Images/226_513.tif  
  inflating: /content/CCAP/content/waipio2022/Images/226_797.tif  
   creating: /content/CCAP/content/waipio2022/Labels/
  inflating: /content/CCAP/content/waipio2022/Labels/1_1.tif  
  inflating: /content/CCAP/content/waipio2022/Labels/1_513.tif  
  inflating: /content/CCAP/content/waipio2022/Labels/1_797.tif  
  inflating: /content/CCAP/content/waipio2022/Labels/226_1.tif  
  inflating: /content/CCAP/content/waipio2022/Labels/226_513.tif  
  inflating: /content/CCAP/content/waipio2022/Labels/226_797.tif  
  inflating: /conte

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [516]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/waipio2022/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/waipio2022/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 6
The number of files in the folder is: 6


In [517]:
df = pd.read_csv('/content/CCAP/content/waipio2022/waipio2022.csv')

In [518]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [519]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
4,513,513,1,1,1,223009.138279,224237.981032,2.229030e+06,2.230259e+06,13,0.000549
5,513,513,1,1,1,223009.138279,224237.981032,2.229030e+06,2.230259e+06,15,0.081520
12,738,513,226,1,2,223549.134298,224777.921548,2.229030e+06,2.230259e+06,13,0.000568
13,738,513,226,1,2,223549.134298,224777.921548,2.229030e+06,2.230259e+06,15,0.163147
20,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,13,0.129986
21,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,14,0.004967
22,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,15,0.134300
28,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,13,0.130367
29,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,14,0.004112
30,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,15,0.137054


In [520]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [521]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
4,513,513,1,1,1,223009.138279,224237.981032,2.229030e+06,2.230259e+06,13,0.000549,1_1.tif
5,513,513,1,1,1,223009.138279,224237.981032,2.229030e+06,2.230259e+06,15,0.081520,1_1.tif
12,738,513,226,1,2,223549.134298,224777.921548,2.229030e+06,2.230259e+06,13,0.000568,226_1.tif
13,738,513,226,1,2,223549.134298,224777.921548,2.229030e+06,2.230259e+06,15,0.163147,226_1.tif
20,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,13,0.129986,1_513.tif
21,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,14,0.004967,1_513.tif
22,513,1025,1,513,3,223009.163311,224237.963071,2.227801e+06,2.229030e+06,15,0.134300,1_513.tif
28,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,13,0.130367,226_513.tif
29,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,14,0.004112,226_513.tif
30,738,1025,226,513,4,223549.106689,224777.924210,2.227801e+06,2.229030e+06,15,0.137054,226_513.tif


In [522]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/waipio2022/Images'
destination_images = '/content/waipio2022/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/waipio2022/Images
Copied 1_1.tif to /content/waipio2022/Images
Copied 226_1.tif to /content/waipio2022/Images
Copied 226_1.tif to /content/waipio2022/Images
Copied 1_513.tif to /content/waipio2022/Images
Copied 1_513.tif to /content/waipio2022/Images
Copied 1_513.tif to /content/waipio2022/Images
Copied 226_513.tif to /content/waipio2022/Images
Copied 226_513.tif to /content/waipio2022/Images
Copied 226_513.tif to /content/waipio2022/Images
Copied 1_797.tif to /content/waipio2022/Images
Copied 1_797.tif to /content/waipio2022/Images
Copied 1_797.tif to /content/waipio2022/Images
Copied 226_797.tif to /content/waipio2022/Images
Copied 226_797.tif to /content/waipio2022/Images
Copied 226_797.tif to /content/waipio2022/Images


In [523]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/waipio2022/Labels'
destination_labels = '/content/waipio2022/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/waipio2022/Labels
Copied 1_1.tif to /content/waipio2022/Labels
Copied 226_1.tif to /content/waipio2022/Labels
Copied 226_1.tif to /content/waipio2022/Labels
Copied 1_513.tif to /content/waipio2022/Labels
Copied 1_513.tif to /content/waipio2022/Labels
Copied 1_513.tif to /content/waipio2022/Labels
Copied 226_513.tif to /content/waipio2022/Labels
Copied 226_513.tif to /content/waipio2022/Labels
Copied 226_513.tif to /content/waipio2022/Labels
Copied 1_797.tif to /content/waipio2022/Labels
Copied 1_797.tif to /content/waipio2022/Labels
Copied 1_797.tif to /content/waipio2022/Labels
Copied 226_797.tif to /content/waipio2022/Labels
Copied 226_797.tif to /content/waipio2022/Labels
Copied 226_797.tif to /content/waipio2022/Labels


In [524]:
# NEW images and labels folder: how many files are there?
new_img = '/content/waipio2022/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/waipio2022/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 6
The number of files in the folder is: 6


In [525]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/waipio2022_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/waipio2022_subset.csv


In [526]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/waipio2022'
zip_file_path = '/content/waipio2022_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/waipio2022_subset.zip


In [527]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/naip'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/naip
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/naip


In [528]:
stopppsps"(W)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-528-871ba97a253f>, line 1)

# Define next folder

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/Kahuku2021.zip -d /content/CCAP/

In [ ]:
!mkdir '/content/kahuku2021'
!mkdir '/content/kahuku2021/Images'
!mkdir '/content/kahuku2021/Labels'

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2021/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2021/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/kahuku2021/kahuku2021.csv')

In [ ]:
df

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2021/Images'
destination_images = '/content/kahuku2021/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2021/Labels'
destination_labels = '/content/kahuku2021/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2021/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2021/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2021_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2021'
zip_file_path = '/content/kahuku2021_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

# one more to do 7, 8, 9 together

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip -d /content/CCAP/

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_009/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_009/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_009/Hawaii_2005_009.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_009/Images'
destination_images = '/content/Hawaii_2005_009/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_009/Labels'
destination_labels = '/content/Hawaii_2005_009/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_009/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_009/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_009_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_009'
zip_file_path = '/content/Hawaii_2005_009_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')